In [1]:
import numpy as np
import pandas as pd 

df = pd.read_csv("../auto_price.csv")
df.head()

,Unnamed: 0,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,0,3,115.0,1,1,1,1,1,1,1,...,130,1,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
1,1,1,115.0,1,1,1,1,2,1,1,...,152,1,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
2,2,2,164.0,2,1,1,2,3,2,1,...,109,1,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
3,3,2,164.0,2,1,1,2,3,3,1,...,136,1,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0
4,4,2,115.0,2,1,1,1,3,2,1,...,136,1,3.19,3.40,8.5,110.0,5500.0,19,25,15250.0


In [37]:
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras import backend as K 

In [2]:
df = df.drop(columns=['Unnamed: 0', 'symboling'])

In [4]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=2)
train, val = train_test_split(train, test_size=0.2, random_state=23)

In [6]:
train_stats = train.describe()
train_stats= train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
normalized-losses,124.0,117.919355,34.602645,65.00,93.75,115.00,130.250,256.00
make,124.0,13.612903,6.246573,1.00,9.00,13.00,20.000,22.00
fuel-type,124.0,1.104839,0.307588,1.00,1.00,1.00,1.000,2.00
aspiration,124.0,1.217742,0.414385,1.00,1.00,1.00,1.000,2.00
num-of-doors,124.0,1.620968,0.487114,1.00,1.00,2.00,2.000,2.00
body-style,124.0,2.822581,0.776153,1.00,2.00,3.00,3.000,5.00
drive-wheels,124.0,1.717742,0.534356,1.00,1.00,2.00,2.000,3.00
engine-location,124.0,1.008065,0.089803,1.00,1.00,1.00,1.000,2.00
wheel-base,124.0,98.679839,5.995527,86.60,94.50,96.50,102.400,115.60
length,124.0,173.137903,12.234754,141.10,166.30,172.40,178.925,202.60


In [7]:
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

In [8]:
train_x = norm(train)
test_x = norm(test)
val_x = norm(val)

In [9]:
train_x.columns 

Index(['normalized-losses', 'make', 'fuel-type', 'aspiration', 'num-of-doors',
       'body-style', 'drive-wheels', 'engine-location', 'wheel-base', 'length',
       'width', 'height', 'curb-weight', 'engine-type', 'num-of-cylinders',
       'engine-size', 'fuel-system', 'bore', 'stroke', 'compression-ratio',
       'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price'],
      dtype='object')

In [10]:
train_x = train_x.drop(columns='price')
test_x = test_x.drop(columns='price')
val_x=val_x.drop(columns='price')

In [11]:
train_y = train['price']
test_y = test['price']
val_y = val['price']

In [13]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

In [ ]:
def square(x):
    return x**2

In [30]:
class SimpleLinear(Layer):

    def __init__(self, units=64, activation=None):
        super(SimpleLinear, self).__init__()
        self.units = units
        self.activation=tf.keras.activations.get(activation)

    def weightsAndBias(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name="kernel",
            initial_value=w_init(shape=(input_shape[-1], self.units),
                                 dtype='float32'),
            trainable=True)

        
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
            initial_value=b_init(shape=(self.units,), dtype='float32'),
            trainable=True)

    def call(self, inputs):
        return self.activation(tf.matmul(inputs, self.w) + self.b)

In [31]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(24,)),
    SimpleLinear(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    SimpleLinear(256, activation='relu'),
    SimpleLinear(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='relu')
])

In [35]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [38]:
model.compile(optimizer='adam',
              loss = rmse,
              metrics=tf.keras.metrics.RootMeanSquaredError())
h = model.fit(train_x, train_y, epochs=20)
model.evaluate(val_x, val_y)

Epoch 1/20
4/4 [==============================] - 0s 3ms/step - loss: 13684.0762 - root_mean_squared_error: 13726.8496
Epoch 2/20
4/4 [==============================] - 0s 3ms/step - loss: 13669.2314 - root_mean_squared_error: 13726.8496
Epoch 3/20
4/4 [==============================] - 0s 3ms/step - loss: 13537.3682 - root_mean_squared_error: 13726.8496
Epoch 4/20
4/4 [==============================] - 0s 3ms/step - loss: 13712.0146 - root_mean_squared_error: 13726.8496
Epoch 5/20
4/4 [==============================] - 0s 3ms/step - loss: 13649.9961 - root_mean_squared_error: 13726.8506
Epoch 6/20
4/4 [==============================] - 0s 3ms/step - loss: 13675.8096 - root_mean_squared_error: 13726.8506
Epoch 7/20
4/4 [==============================] - 0s 3ms/step - loss: 13663.0381 - root_mean_squared_error: 13726.8389
Epoch 8/20
4/4 [==============================] - 0s 2ms/step - loss: 13690.4902 - root_mean_squared_error: 13726.1270
Epoch 9/20
4/4 [==============================] 

[12764.6083984375, 12764.609375]

In [65]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(24,)),
    SimpleLinear(512),
    tf.keras.layers.Lambda(lambda x: tf.abs(x)),
    tf.keras.layers.Dropout(0.2),
    SimpleLinear(256),
    tf.keras.layers.Lambda(lambda x: tf.abs(x)),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: tf.abs(x)),
])

In [44]:
model.compile(optimizer='adam',
              loss = rmse,
              metrics=tf.keras.metrics.RootMeanSquaredError())
h = model.fit(train_x, train_y, epochs=3)
model.evaluate(val_x, val_y)

Epoch 1/3
4/4 [==============================] - 0s 7ms/step - loss: 13663.2793 - root_mean_squared_error: 13724.6494
Epoch 2/3
4/4 [==============================] - 0s 3ms/step - loss: 13713.2539 - root_mean_squared_error: 13719.8193
Epoch 3/3
1/1 [==============================] - 0s 63ms/step - loss: 15765.9355 - root_mean_squared_error: 15765.9355


[15765.935546875, 15765.935546875]

In [45]:
def active1(x):
    return x**2

In [46]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(24,)),
    SimpleLinear(512),
    tf.keras.layers.Lambda(active1),
    tf.keras.layers.Dropout(0.2),
    SimpleLinear(256),
    tf.keras.layers.Lambda(active1),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(active1),
])

In [49]:
model.compile(optimizer='adam',
              loss = rmse,
              metrics=tf.keras.metrics.RootMeanSquaredError())
h = model.fit(train_x, train_y, epochs=10)
model.evaluate(val_x, val_y)

Epoch 1/10
4/4 [==============================] - 0s 2ms/step - loss: 13487.8037 - root_mean_squared_error: 13504.3154
Epoch 2/10
4/4 [==============================] - 0s 3ms/step - loss: 12651.0527 - root_mean_squared_error: 12656.0117
Epoch 3/10
4/4 [==============================] - 0s 3ms/step - loss: 12887.9980 - root_mean_squared_error: 12918.5605
Epoch 4/10
4/4 [==============================] - 0s 2ms/step - loss: 12398.4980 - root_mean_squared_error: 12459.9727
Epoch 5/10
4/4 [==============================] - 0s 2ms/step - loss: 12128.4258 - root_mean_squared_error: 12138.7061
Epoch 6/10
4/4 [==============================] - 0s 3ms/step - loss: 12129.1045 - root_mean_squared_error: 12167.2744
Epoch 7/10
4/4 [==============================] - 0s 3ms/step - loss: 12285.8691 - root_mean_squared_error: 12321.1221
Epoch 8/10
4/4 [==============================] - 0s 2ms/step - loss: 12079.3867 - root_mean_squared_error: 12100.0439
Epoch 9/10
4/4 [==============================] 

[14131.955078125, 14131.955078125]

In [48]:
def active2(x):
    return K.maximum(-0.1, x)

In [50]:
def active3(x):
    x -= K.mean(x, axis=1, keepdims=True)
    x = K.l2_normalize(x, axis=1)
    pos=K.relu(x)
    neg=K.relu(-x)
    return K.concatenate([pos, neg], axis=1)

In [51]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(24,)),
    SimpleLinear(512),
    tf.keras.layers.Lambda(active3),
    tf.keras.layers.Dropout(0.2),
    SimpleLinear(256),
    tf.keras.layers.Lambda(active3),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(active3),
])

In [52]:
model.compile(optimizer='adam',
              loss = rmse,
              metrics=tf.keras.metrics.RootMeanSquaredError())
h = model.fit(train_x, train_y, epochs=3)
model.evaluate(val_x, val_y)

Epoch 1/3
4/4 [==============================] - 0s 4ms/step - loss: 13708.4746 - root_mean_squared_error: 13726.8506
Epoch 2/3
4/4 [==============================] - 0s 3ms/step - loss: 13665.5850 - root_mean_squared_error: 13726.8496
Epoch 3/3
1/1 [==============================] - 0s 83ms/step - loss: 15786.0742 - root_mean_squared_error: 15786.0742


[15786.07421875, 15786.07421875]

In [56]:
def linear_transformation(x):
    w = 1.
    b = 0.
    return x*w+b

In [57]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(24,)),
    SimpleLinear(512),
    tf.keras.layers.Lambda(linear_transformation),
    tf.keras.layers.Dropout(0.2),
    SimpleLinear(256),
    tf.keras.layers.Lambda(linear_transformation),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(linear_transformation),
])

In [59]:
model.compile(optimizer='adam',
              loss = rmse,
              metrics=tf.keras.metrics.RootMeanSquaredError())
h = model.fit(train_x, train_y, epochs=3)
model.evaluate(val_x, val_y)

Epoch 1/3
4/4 [==============================] - 0s 3ms/step - loss: 13644.5283 - root_mean_squared_error: 13726.1318
Epoch 2/3
4/4 [==============================] - 0s 3ms/step - loss: 13719.1416 - root_mean_squared_error: 13723.9922
Epoch 3/3
1/1 [==============================] - 0s 85ms/step - loss: 15775.0791 - root_mean_squared_error: 15775.0781


[15775.0791015625, 15775.078125]

In [60]:
def output_of_lambda(input_shape):
    return (input_shape[0], 1, input_shape[1])

def mean(x):
    return K.mean(x, axis=1, keepdims=True)

In [61]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(24,)),
    SimpleLinear(512),
    tf.keras.layers.Lambda(mean, output_shape=output_of_lambda),
    tf.keras.layers.Dropout(0.2),
    SimpleLinear(256),
    tf.keras.layers.Lambda(mean, output_shape=output_of_lambda),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(mean, output_shape=output_of_lambda),
])

In [62]:
model.compile(optimizer='adam',
              loss = rmse,
              metrics=tf.keras.metrics.RootMeanSquaredError())
h = model.fit(train_x, train_y, epochs=3)
model.evaluate(val_x, val_y)

Epoch 1/3
4/4 [==============================] - 0s 2ms/step - loss: 13710.7480 - root_mean_squared_error: 13726.8467
Epoch 2/3
4/4 [==============================] - 0s 2ms/step - loss: 13654.8828 - root_mean_squared_error: 13726.8389
Epoch 3/3
1/1 [==============================] - 0s 59ms/step - loss: 15786.0479 - root_mean_squared_error: 15786.0479


[15786.0478515625, 15786.0478515625]